In [ ]:
!pip install torchinfo

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict
import torch
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from torchsummary import summary
from torchinfo import summary
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import wandb
import torch.nn.functional as F
import hashlib
from typing import Dict, Tuple
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
config = {
    'batch_size': 64,            # Increased batch size for better gradient estimates
    'lr': 0.001,                 # Learning rate
    'weight_decay': 1e-4,        # L2 regularization
    'epochs': 10,                # More epochs with early stopping
    'input_dim': 22,             # Your feature dimension
    'num_classes': 7,            # Number of object classes
    'hidden_dims': [512,256,128],           # Hidden dimension
    'num_layers': 2,             # Number of LSTM layers
    'dropout': 0.3,              # Dropout rate
    'patience': 7,               # Early stopping patience
    'seq_len': 2500,             # Your sequence length
    'checkpoint_dir': "/content/drive/MyDrive/IDL/Checkpoint",
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("Device: ", device)

Device:  cuda


In [ ]:
# Define category mapping
CATEGORIES = {
    'Blueball': 0,
    'Box': 1,
    'Pencilcase': 2,
    'Pinkball': 3,
    'StuffedAnimal': 4,
    'Tennis': 5,
    'Waterbottle': 6,
}

In [ ]:
# Path to the folder containing the dataset files
folder_path = "/content/drive/MyDrive/IDL/IDL_Data"

In [ ]:
# Stats trackers
total_count = 0
kept_count = 0
valid_file_count = 0
skipped_due_to_missing_waypoints = 0

In [ ]:
WAYPOINTS = [
    (30, -30), (30, 30), (15, -30), (15, 30),
    (0, -30), (0, 30), (-15, -30), (-15, 30),
    (-30, -30), (-30, 30), (-30, -30), (30, -30),
    (30, 30), (-30, 30)
]


In [ ]:
# Step 1: Load and label dataset
def load_and_label_file(file_path, file_name):
    global total_count
    category = next((key for key in CATEGORIES if key in file_name), None)
    if category is None:
        return pd.DataFrame()

    data = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 10:
                try:
                    timestamp = parts[0]
                    microsec = int(parts[1])
                    x = float(parts[2])
                    y = float(parts[3])
                    x_target = float(parts[4])
                    y_target = float(parts[5])
                    pwm1 = int(parts[6])
                    pwm2 = int(parts[7])
                    pwm3 = int(parts[8])
                    pwm4 = int(parts[9])
                    total_count += 1

                    data.append([
                        timestamp, microsec, x, y, x_target, y_target,
                        pwm1, pwm2, pwm3, pwm4, category, CATEGORIES[category]
                    ])
                except ValueError:
                    continue

    return pd.DataFrame(data, columns=[
        "timestamp", "microseconds", "x", "y", "x_target", "y_target",
        "pwm1", "pwm2", "pwm3", "pwm4", "category", "label"
    ])

In [ ]:
# Step 2: Assign sequential waypoint numbers
def assign_sequential_waypoints(df, tol=1.0):
    df = df.reset_index(drop=True)
    wp_index = 0
    assigned_wp = []

    for i in range(len(df)):
        x_t, y_t = df.loc[i, "x_target"], df.loc[i, "y_target"]
        current_expected = WAYPOINTS[wp_index]

        if np.isclose(x_t, current_expected[0], atol=tol) and np.isclose(y_t, current_expected[1], atol=tol):
            assigned_wp.append(wp_index)
        else:
            if wp_index + 1 < len(WAYPOINTS):
                next_expected = WAYPOINTS[wp_index + 1]
                if np.isclose(x_t, next_expected[0], atol=tol) and np.isclose(y_t, next_expected[1], atol=tol):
                    wp_index += 1
                    assigned_wp.append(wp_index)
                else:
                    assigned_wp.append(wp_index)
            else:
                assigned_wp.append(wp_index)

    df["waypoint_number"] = assigned_wp
    return df

In [ ]:
# Step 3: Filter out rows where y <= 0
def filter_by_y(df):
    global kept_count
    filtered = df[df["y"] > 15].reset_index(drop=True)
    kept_count += len(filtered)
    return filtered

In [ ]:
def process_file(file_path, file_name):
    global valid_file_count

    # Step 1: Load and label
    df = load_and_label_file(file_path, file_name)
    if df.empty:
        return pd.DataFrame()

    # Step 2: Assign waypoint numbers
    df = assign_sequential_waypoints(df)

    # Show how many waypoints existed before filtering
    waypoint_count_before = df["waypoint_number"].nunique()
    print(f"\n{file_name} → {waypoint_count_before} waypoints BEFORE filtering")

    # Step 3: Filter out rows where y ≤ 0
    df = filter_by_y(df)

    # Show how many remain after filtering
    waypoint_count_after = df["waypoint_number"].nunique()
    print(f"{file_name} → {waypoint_count_after} waypoints AFTER filtering")

    # Count as valid if any data was kept
    if not df.empty:
        valid_file_count += 1

    return df

In [ ]:
# print(all_data.head(1350))

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
from collections import defaultdict, Counter

def build_datasets(data_dir: str):
    data_dir = Path(data_dir)
    file_paths = list(data_dir.glob("*.txt"))
    random.seed(42)

    class_to_files = defaultdict(list)
    for file_path in file_paths:
        for class_name in CATEGORIES:
            if class_name in file_path.name:
                class_to_files[class_name].append(file_path)
                break
    train_files, val_files, test_files = [], [], []
    for class_name, files in class_to_files.items():
        random.shuffle(files)
        n = len(files)
        train_split = int(0.65 * n)
        val_split = int(0.85 * n)
        train_files += files[:train_split]
        val_files += files[train_split:val_split]
        test_files += files[val_split:]

    print("Per-class file counts:")
    for cls in CATEGORIES:
        print(f"  {cls:<15} → {len(class_to_files[cls])} total files")

    print("Final split file counts:")
    print(f"Train: {len(train_files)}")
    print(f"Val:   {len(val_files)}")
    print(f"Test:  {len(test_files)}")

    def process_file_list(file_list):
        dfs = []
        for fp in file_list:
            df = process_file(fp, fp.name)
            if not df.empty:
                dfs.append(df)
        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

    print("Processing train files")
    train_df = process_file_list(train_files)
    print("Processing validation files")
    val_df = process_file_list(val_files)
    print("Processing test files")
    test_df = process_file_list(test_files)

    return train_df, val_df, test_df


# Run everything
train_df, val_df, test_df = build_datasets(folder_path)

🔍 Per-class file counts:
  Blueball        → 30 total files
  Box             → 30 total files
  Pencilcase      → 29 total files
  Pinkball        → 30 total files
  StuffedAnimal   → 30 total files
  Tennis          → 30 total files
  Waterbottle     → 30 total files

✅ Final split file counts:
Train: 132
Val:   42
Test:  35
Processing train files

📌 StuffedAnimal21.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal21.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal24.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal24.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal25.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal25.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal2.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal2.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal13.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal13.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal28.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal28.txt → 12 waypoints AFTER filtering

📌

In [ ]:
def extract_enhanced_features(df):
    """Extract additional meaningful features from the raw sensor data"""
    # Create a copy to avoid modifying the original
    enhanced_df = df.copy()

    # Compute deltas (changes between consecutive measurements)
    enhanced_df['x_delta'] = enhanced_df.groupby(['category', 'waypoint_number'])['x'].diff().fillna(0)
    enhanced_df['y_delta'] = enhanced_df.groupby(['category', 'waypoint_number'])['y'].diff().fillna(0)

    # Distance to target (error signal that drives the controller)
    enhanced_df['x_error'] = enhanced_df['x_target'] - enhanced_df['x']
    enhanced_df['y_error'] = enhanced_df['y_target'] - enhanced_df['y']

    # Magnitude of error and movement
    enhanced_df['error_magnitude'] = np.sqrt(enhanced_df['x_error']**2 + enhanced_df['y_error']**2)
    enhanced_df['movement_magnitude'] = np.sqrt(enhanced_df['x_delta']**2 + enhanced_df['y_delta']**2)

    # Control effort features (sum and differences of PWM signals)
    enhanced_df['total_pwm'] = enhanced_df['pwm1'] + enhanced_df['pwm2'] + enhanced_df['pwm3'] + enhanced_df['pwm4']
    enhanced_df['pwm_x_diff'] = enhanced_df['pwm1'] - enhanced_df['pwm3']  # Assuming these control x-axis
    enhanced_df['pwm_y_diff'] = enhanced_df['pwm2'] - enhanced_df['pwm4']  # Assuming these control y-axis

    # Interaction features (product of error and control)
    enhanced_df['x_control_response'] = enhanced_df['x_error'] * enhanced_df['pwm_x_diff']
    enhanced_df['y_control_response'] = enhanced_df['y_error'] * enhanced_df['pwm_y_diff']

    # Time derivatives of error (how quickly error is changing)
    enhanced_df['x_error_delta'] = enhanced_df.groupby(['category', 'waypoint_number'])['x_error'].diff().fillna(0)
    enhanced_df['y_error_delta'] = enhanced_df.groupby(['category', 'waypoint_number'])['y_error'].diff().fillna(0)

    return enhanced_df

# Process your dataframes with the enhanced features
train_df_enhanced = extract_enhanced_features(train_df)
val_df_enhanced = extract_enhanced_features(val_df)
test_df_enhanced = extract_enhanced_features(test_df)

In [ ]:
class EnhancedWindowedDataset(torch.utils.data.Dataset):
    def __init__(self, df, seq_len=2000, augment=False, noise_scale=0.02, time_warp_scale=0.1):
        self.seq_len = seq_len
        self.df = df.reset_index(drop=True)  # Use the enhanced dataframe
        self.augment = augment
        self.noise_scale = noise_scale
        self.time_warp_scale = time_warp_scale

        # Select all features including the new engineered features
        feature_columns = [
            # Original features
            "x", "y", "x_target", "y_target", "pwm1", "pwm2", "pwm3", "pwm4", "waypoint_number",
            # New engineered features
            "x_delta", "y_delta", "x_error", "y_error", "error_magnitude", "movement_magnitude",
            "total_pwm", "pwm_x_diff", "pwm_y_diff", "x_control_response", "y_control_response",
            "x_error_delta", "y_error_delta"
        ]

        # Filter to only include columns that exist in the dataframe
        self.feature_columns = [col for col in feature_columns if col in self.df.columns]
        self.features = self.df[self.feature_columns].values.astype(np.float32)

        # Normalize features
        self.feature_means = np.mean(self.features, axis=0)
        self.feature_stds = np.std(self.features, axis=0) + 1e-6  # Avoid division by zero
        self.features = (self.features - self.feature_means) / self.feature_stds

        # Label per row
        self.labels = self.df["label"].values.astype(np.int64)

    def __len__(self):
        return len(self.df) - self.seq_len + 1

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.seq_len]  # (seq_len, input_dim)
        y = self.labels[idx + self.seq_len - 1]

        # Apply augmentation if enabled
        if self.augment and np.random.random() > 0.5:
            # Get dimensions once at the start
            seq_len, feat_dim = x.shape

            # Add random noise
            noise = np.random.normal(0, self.noise_scale, x.shape)
            x = x + noise

            # Time warping (randomly stretch or compress parts of the sequence)
            if np.random.random() > 0.7:
                time_indices = np.arange(seq_len)
                warp = np.sin(np.linspace(0, 3*np.pi, seq_len)) * self.time_warp_scale
                warped_indices = np.clip(time_indices + warp, 0, seq_len-1).astype(int)
                x = x[warped_indices, :]

            # Random feature masking (occasionally zero out features)
            if np.random.random() > 0.8:
                mask_idx = np.random.choice(feat_dim, size=int(feat_dim * 0.1), replace=False)
                x[:, mask_idx] = 0

        x_tensor = torch.tensor(x, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        return x_tensor, y_tensor

In [ ]:
seq_len = config['seq_len']  # Adjust based on your needs

# Create datasets with augmentation
train_dataset = EnhancedWindowedDataset(train_df_enhanced, seq_len=seq_len, augment=True)
val_dataset = EnhancedWindowedDataset(val_df_enhanced, seq_len=seq_len)
test_dataset = EnhancedWindowedDataset(test_df_enhanced, seq_len=seq_len)

# Create data loaders with smaller batch size
batch_size = 16  # Smaller batch size for better regularization
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self, input_channels, seq_len):
        super(CNNFeatureExtractor, self).__init__()

        # CNN layers for extracting temporal patterns
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(256)

        # Pooling to reduce dimensionality
        self.pool = nn.MaxPool1d(kernel_size=4, stride=4)

        # Calculate output size after convolutions and pooling
        output_len = seq_len // 16  # After 2 pooling layers (4×4=16)
        self.output_size = 256 * output_len if output_len > 0 else 256

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_channels]
        # Convert to [batch_size, input_channels, seq_len] for CNN
        x = x.permute(0, 2, 1)

        # Apply CNN layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))

        # Flatten for MLP
        x = x.view(x.size(0), -1)
        return x

class StatisticalFeatureExtractor(nn.Module):
    def __init__(self, seq_len, input_dim):
        super(StatisticalFeatureExtractor, self).__init__()
        self.seq_len = seq_len
        self.input_dim = input_dim

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]

        # Calculate statistical features
        mean = torch.mean(x, dim=1)  # Mean across sequence
        std = torch.std(x, dim=1)    # Standard deviation
        max_vals, _ = torch.max(x, dim=1)  # Maximum values
        min_vals, _ = torch.min(x, dim=1)  # Minimum values

        # Calculate deltas (changes between timesteps)
        deltas = x[:, 1:, :] - x[:, :-1, :]
        mean_deltas = torch.mean(deltas, dim=1)
        std_deltas = torch.std(deltas, dim=1)

        # Concatenate all features
        features = torch.cat([mean, std, max_vals, min_vals, mean_deltas, std_deltas], dim=1)
        return features

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.3):
        super(MLPClassifier, self).__init__()

        layers = []
        prev_dim = input_dim

        # Create hidden layers
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim

        # Output layer
        layers.append(nn.Linear(prev_dim, num_classes))

        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)

class TactileEnsembleModel(nn.Module):
    def __init__(self, input_dim, seq_len, num_classes, hidden_dims=[512, 256, 128], dropout=0.3):
        super(TactileEnsembleModel, self).__init__()

        # CNN feature extractor
        self.cnn_extractor = CNNFeatureExtractor(input_dim, seq_len)
        cnn_output_size = self.cnn_extractor.output_size

        # Statistical feature extractor
        self.stat_extractor = StatisticalFeatureExtractor(seq_len, input_dim)
        stat_output_size = input_dim * 6  # mean, std, max, min, mean_delta, std_delta

        # Combined feature size
        combined_size = cnn_output_size + stat_output_size

        # MLP classifier
        self.classifier = MLPClassifier(combined_size, hidden_dims, num_classes, dropout)

        # Regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Extract CNN features
        cnn_features = self.cnn_extractor(x)

        # Extract statistical features
        stat_features = self.stat_extractor(x)

        # Combine features
        combined = torch.cat([cnn_features, stat_features], dim=1)
        combined = self.dropout(combined)

        # Final classification
        logits = self.classifier(combined)

        # Return both logits and softmax probabilities
        return logits, F.softmax(logits, dim=1)

model = TactileEnsembleModel(
    input_dim=config['input_dim'],
    seq_len=config['seq_len'],
    num_classes=config['num_classes'],
    hidden_dims=config['hidden_dims'],
    dropout=config['dropout']
).to(config['device'])

summary(model, input_size=(32, config['seq_len'], config['input_dim']))

Layer (type:depth-idx)                   Output Shape              Param #
TactileEnsembleModel                     [32, 7]                   --
├─CNNFeatureExtractor: 1-1               [32, 39936]               --
│    └─Conv1d: 2-1                       [32, 64, 2500]            7,104
│    └─BatchNorm1d: 2-2                  [32, 64, 2500]            128
│    └─MaxPool1d: 2-3                    [32, 64, 625]             --
│    └─Conv1d: 2-4                       [32, 128, 625]            41,088
│    └─BatchNorm1d: 2-5                  [32, 128, 625]            256
│    └─MaxPool1d: 2-6                    [32, 128, 156]            --
│    └─Conv1d: 2-7                       [32, 256, 156]            164,096
│    └─BatchNorm1d: 2-8                  [32, 256, 156]            512
├─StatisticalFeatureExtractor: 1-2       [32, 132]                 --
├─Dropout: 1-3                           [32, 40068]               --
├─MLPClassifier: 1-4                     [32, 7]                   --


In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def train_model_with_regularization(model, train_loader, criterion, optimizer, device,
                                   weight_decay=1e-4, gradient_clip=1.0, mixup_alpha=0.2):
    model.train()
    loss_m = AverageMeter()
    acc_m = AverageMeter()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        x, y = data
        batch_size = x.size(0)

        # Move data to device first
        x, y = x.to(device), y.to(device)

        # Apply Mixup (data augmentation technique)
        if mixup_alpha > 0:
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            index = torch.randperm(batch_size).to(device)
            x = lam * x + (1 - lam) * x[index]

        # Forward pass - now returns logits and probabilities
        logits, probs = model(x)

        # Compute loss using logits (not probabilities)
        if mixup_alpha > 0:
            loss = lam * criterion(logits, y) + (1 - lam) * criterion(logits, y[index])
        else:
            loss = criterion(logits, y)

        # Add L2 regularization explicitly if needed
        l2_reg = torch.tensor(0., device=device)
        for param in model.parameters():
            l2_reg += torch.norm(param, 2)
        loss += weight_decay * l2_reg

        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip)
        optimizer.step()

        # Compute accuracy using probabilities
        _, predicted = torch.max(probs.data, 1)
        correct = (predicted == y).sum().item()
        acc = 100.0 * correct / batch_size

        loss_m.update(loss.item())
        acc_m.update(acc)

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg)),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr']))
        )
        batch_bar.update()

        del x, y, logits, probs, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    return loss_m.avg, acc_m.avg

In [ ]:
@torch.no_grad()
def validate_model(model, val_loader, criterion, class_names, device):
    model.eval()
    loss_m = AverageMeter()
    acc_m = AverageMeter()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    all_preds = []
    all_targets = []

    for i, data in enumerate(val_loader):
        x, y = data
        x, y = x.to(device), y.to(device)

        # Forward pass - now returns logits and probabilities
        logits, probs = model(x)

        # Compute loss using logits
        loss = criterion(logits, y)

        # Get predictions from probabilities
        _, predicted = torch.max(probs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

        # Calculate accuracy
        correct = (predicted == y).sum().item()
        acc = 100.0 * correct / y.size(0)

        loss_m.update(loss.item())
        acc_m.update(acc)

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg))
        )
        batch_bar.update()

        del x, y, logits, probs, loss
        torch.cuda.empty_cache()

    batch_bar.close()

    if class_names:
        print("\nPer-class Validation Accuracy:")
        per_class_acc = {}
        for i, class_name in enumerate(class_names):
            class_mask = (np.array(all_targets) == i)
            if np.sum(class_mask) > 0:
                class_correct = np.sum((np.array(all_preds)[class_mask] == i))
                class_total = np.sum(class_mask)
                acc_percent = 100 * class_correct / class_total
                print(f"  {class_name}: {acc_percent:.4f}% ({class_correct}/{class_total})")
                per_class_acc[f"val_acc_{class_name}"] = acc_percent

    return loss_m.avg, acc_m.avg

In [ ]:
def save_model(model, optimizer, scheduler, metrics, epoch, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'metrics': metrics
    }, path)

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Use AdamW optimizer with weight decay
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['lr'],
    weight_decay=config['weight_decay']
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=True
)

# Early stopping initialization
early_stopping = EarlyStopping(
    patience=7,
    verbose=True,
    path=os.path.join(config['checkpoint_dir'], 'best_model.pth')
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name="CNN+MLP Ensemble",
    project="object_classification",
    config={
        'architecture': 'BiLSTM-Attention',
        'input_dim': config['input_dim'],
        'seq_len': config['seq_len'],
        'batch_size': config['batch_size'],
        'learning_rate': config['lr'],
        'weight_decay': config['weight_decay'],
        'dropout': config['dropout'],
        'hidden_dims': config['hidden_dims'],
        'num_layers': config['num_layers']
    }
)

In [ ]:
best_val_loss = float('inf')
best_val_acc = 0
epochs = 3
class_names = list(CATEGORIES.keys())

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Train
    train_loss, train_acc = train_model_with_regularization(
        model,
        train_loader,
        criterion,
        optimizer,
        device,
        weight_decay=1e-3,
        gradient_clip=1.0,
        mixup_alpha=0.2
    )
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    # Validate
    val_loss, val_acc = validate_model(model, val_loader, criterion, class_names, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    # Update learning rate
    scheduler.step(val_loss)

    # Early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Save checkpoint
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_val_loss = val_loss
        print(f"Saved best model with validation loss: {best_val_loss:.4f} and accuracy: {best_val_acc:.2f}%")

    # Log metrics to wandb
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'learning_rate': optimizer.param_groups[0]['lr']
    })

    print(f"End of Epoch {epoch+1}/{epochs}")

print(f"\nTraining complete! Best validation accuracy: {best_val_acc:.2f}%")


Epoch 1/3


Train Loss: 0.8845, Train Accuracy: 59.02%



Per-class Validation Accuracy:
  Blueball: 68.8592% (15609/22668)
  Box: 65.2853% (14210/21766)
  Pencilcase: 44.8167% (9913/22119)
  Pinkball: 57.2613% (11529/20134)
  StuffedAnimal: 35.4703% (6938/19560)
  Tennis: 60.7053% (13978/23026)
  Waterbottle: 45.6078% (10119/22187)
Validation Loss: 1.6734, Validation Accuracy: 54.33%
EarlyStopping counter: 1 out of 7
Saved best model with validation loss: 1.6734 and accuracy: 54.33%


Error: You must call wandb.init() before wandb.log()

In [ ]:
@torch.no_grad()
def test_model(model, test_loader, criterion, class_names, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    all_probs = []

    class_correct = {class_name: 0 for class_name in class_names}
    class_total = {class_name: 0 for class_name in class_names}

    for data in test_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass - now returns logits and probabilities
        logits, probs = model(inputs)

        # Compute loss using logits
        loss = criterion(logits, targets)
        test_loss += loss.item() * inputs.size(0)

        # Get predictions from probabilities
        _, predicted = torch.max(probs, 1)

        total += targets.size(0)
        correct += (predicted == targets).sum().item()

        for i in range(targets.size(0)):
            label = targets[i].item()
            pred = predicted[i].item()
            class_name = class_names[label]
            class_total[class_name] += 1
            if pred == label:
                class_correct[class_name] += 1

        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

    test_loss /= len(test_loader.dataset)
    test_acc = correct / total

    class_accuracy = {
        name: class_correct[name]/class_total[name] if class_total[name] > 0 else 0
        for name in class_names
    }

    print("\n" + "="*50)
    print("TEST RESULTS")
    print("="*50)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f} ({correct}/{total})")
    print("\nPer-Class Accuracy:")
    for class_name in class_names:
        print(f" {class_name}: {class_accuracy[class_name]:.4f} ({class_correct[class_name]}/{class_total[class_name]})")

    return {
        'test_loss': test_loss,
        'test_accuracy': test_acc,
        'class_accuracy': class_accuracy,
        'predictions': all_preds,
        'targets': all_targets,
        'probabilities': all_probs
    }

In [ ]:
# model.load_state_dict(torch.load(os.path.join(config['checkpoint_dir'], 'best_model.pth')))

# Test the model
test_results = test_model(model, test_loader, criterion, class_names, device)

# Log final test results to wandb
wandb.log({
    'test_loss': test_results['test_loss'],
    'test_accuracy': test_results['test_accuracy'],
    'confusion_matrix': wandb.plot.confusion_matrix(
        probs=None,
        y_true=test_results['targets'],
        preds=test_results['predictions'],
        class_names=class_names
    )
})

# Finish wandb run
wandb.finish()

print(f"Final Test Accuracy: {test_results['test_accuracy']:.2f}%")

Train:   8%|▊         | 2492/30809 [02:08<24:05, 19.59it/s, acc=58.9812%, loss=0.9027, lr=0.001000]


TEST RESULTS
Test Loss: 1.7264
Test Accuracy: 0.5286 (69905/132234)

Per-Class Accuracy:
 Blueball: 0.5231 (9085/17368)
 Box: 0.2100 (3738/17804)
 Pencilcase: 0.4521 (10100/22339)
 Pinkball: 0.9693 (17471/18024)
 StuffedAnimal: 0.7700 (14789/19207)
 Tennis: 0.3635 (7163/19703)
 Waterbottle: 0.4249 (7559/17789)


epoch,▁█
learning_rate,▁▁
test_accuracy,▁
test_loss,▁
train_acc,▁█
train_loss,█▁
val_acc,▁█
val_loss,█▁
epoch,2
learning_rate,0.001
test_accuracy,0.52865


Final Test Accuracy: 0.53%


In [ ]:
# You can visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(conf_matrix, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Plot the confusion matrix
if detailed_results:
    plot_confusion_matrix(detailed_results['confusion_matrix'], class_names)